In [286]:
import pandas as pd
import numpy as np
import math

In [288]:
df = pd.read_csv('Software Engineer Salaries.csv')

In [290]:
df.head()

,Company,Company Score,Job Title,Location,Date,Salary
0,ViewSoft,4.8,Software Engineer,"Manassas, VA",8d,$68K - $94K (Glassdoor est.)
1,Workiva,4.3,Software Support Engineer,Remote,2d,$61K - $104K (Employer est.)
2,"Garmin International, Inc.",3.9,C# Software Engineer,"Cary, NC",2d,$95K - $118K (Glassdoor est.)
3,Snapchat,3.5,"Software Engineer, Fullstack, 1+ Years of Expe...","Los Angeles, CA",2d,$97K - $145K (Employer est.)
4,Vitesco Technologies Group AG,3.1,Software Engineer,"Seguin, TX",2d,$85K - $108K (Glassdoor est.)


In [292]:
def split_salary_col(val):
    data_source = ""
    pay = ""
    
    # Check for null and nan
    if val == None:
        val = ""

    # Convert value to lower case
    val = str(val)
    val = val.lower().replace("$", "")

    # Extract the "source" part of the value
    if val.find("(glassdoor est.)") != -1:
        data_source = "Glassdoor"
    elif val.find("(employer est.)") != -1:
        data_source = "Employer"
    else:
        data_source = "Unknown"

    # Extract the "pay per hour vs. salary" part of the value
    if val.find("per hour") != -1:
        pay = "hourly"
    else:
        pay = "salary"

    # Remove "source" part of the value
    val = str(val).replace("(glassdoor est.)", "").replace("(employer est.)", "").strip()

    # Remove "pay" part of the value
    val = val.replace("per hour", "")

    # Replace instances of letter 'k' with '000' to signify thousands
    val = val.replace("k", "000")
    
    # Split value into lower and upper range bounds
    temp = val.split("-")

    # Check if only the lower range exists.  If only lower range exists, duplicate it to upper range
    if len(temp) == 1:
        temp.append(temp[0])

    # Deal with blank or missing or NULL values
    for i in range(0, len(temp)):
        temp[i] = temp[i].strip().replace("nan", "0")
        if temp[i] == "":
            temp[i] = 0
    
    
    return data_source, pay, float(temp[0]), float(temp[1])
    

In [294]:
# Test the function
split_salary_col(df['Salary'].loc[0])

('Glassdoor', 'salary', 68000.0, 94000.0)

In [296]:
# Test the function more
split_salary_col("65K")

('Unknown', 'salary', 65000.0, 65000.0)

In [298]:
# Test the function even more with null values
split_salary_col(None)

('Unknown', 'salary', 0.0, 0.0)

In [300]:
# And now test with NaN values
split_salary_col(np.nan)

('Unknown', 'salary', 0.0, 0.0)

In [302]:
# Create new columns and initialize the values
df[['DataSource', 'Pay']] = ''
df[['IncomeLowerBound', 'IncomeUpperBound']] = 0.0

In [304]:
# Iterate through the DataFrame
for idx, row in df.iterrows():
    data = split_salary_col(row['Salary']) # This will return four values (employer, pay, lower range, upper range)
    df.at[idx, 'DataSource'] = data[0] # Data source refers to who reported the income - employer vs. glassdoor
    df.at[idx, 'Pay'] = data[1] # Hourly vs. salary
    df.at[idx, 'IncomeLowerBound'] = data[2] # Lower bound of the pay range
    df.at[idx, 'IncomeUpperBound'] = data[3] # Upper bound of the pay range

In [310]:
df.head()

,Company,Company Score,Job Title,Location,Date,Salary,DataSource,Pay,IncomeLowerBound,IncomeUpperBound
86,ILOGIC INC,3.5,SOFTWARE DEVELOPERS,Township of Hamilton,30d+,$80.00 Per Hour (Employer est.),Employer,hourly,80.0,80.0
115,Two Six Technologies,4.1,Software Engineer,"Warrenton, VA",17d,$50.00 Per Hour (Employer est.),Employer,hourly,50.0,50.0
161,Mailprotector,4.6,Software Engineer,Remote,2d,$50.00 Per Hour (Employer est.),Employer,hourly,50.0,50.0
174,"First Command Financial Services, Inc.",4.2,Software Engineer,NaN,2d,$50.00 - $70.00 Per Hour (Employer est.),Employer,hourly,50.0,70.0
212,Salesforce,3.1,Software Engineering MTS,"San Francisco, CA",3d,$40.00 - $55.00 Per Hour (Employer est.),Employer,hourly,40.0,55.0
